In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 14.9 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.8 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039298 sha256=20b2bb9a48104ff09cfb1dcc9d455da9e887e4ed32a647f51824c6dbb64407c1
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for umap-l

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)

    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]

    return ' '.join(filtered_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
import os

base_path = '/content/drive/MyDrive/bbc/'
# topics = ['tech','politics','sport','business','entertainment']
topics = ['politics']
contents = {}
for topic in topics:
  contents[topic] = []
  folder_path = base_path + topic
  # Get a list of all files in the folder
  all_files = os.listdir(folder_path)
  # Filter out only the files (excluding subdirectories)
  files_only = [file for file in all_files if os.path.isfile(os.path.join(folder_path, file))]
  for file in files_only:
      file_path = os.path.join(folder_path, file)
      with open(file_path, 'r', encoding='utf-8') as txt_file:
            file_content = txt_file.read()
            contents[topic].append(remove_stopwords(file_content))


In [10]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

top_topics = {}
for topic in topics:
  # Create BERTopic model
  model = BERTopic()
  topics, probabilities = model.fit_transform(contents[topic])
  # Get the top topics
  top_topic = model.get_topic_freq().head(5)  # Get the top 5 topics
  top_topics[topic] = {"topics":top_topic, "model":model}

In [11]:
politics = top_topics['politics']
# Print the top topics
print(politics["topics"])
politics["model"].get_topic_info()
# Visualize topics
# politics["model"].visualize_topics()

   Topic  Count
0     -1    114
5      0     59
1      1     58
2      2     49
3      3     18


,Topic,Count,Name,Representation,Representative_Docs
0,-1,114,-1_said_mr_would_people,"[said, mr, would, people, labour, government, ...",[Howard denies split ID cards Michael Howard d...
1,0,59,0_mr_blair_labour_brown,"[mr, blair, labour, brown, election, minister,...",[Blair dismisses quit claim report Tony Blair ...
2,1,58,1_tax_labour_would_election,"[tax, labour, would, election, mr, brown, said...",[Howard dismisses Tory tax fears Michael Howar...
3,2,49,2_said_police_law_would,"[said, police, law, would, rights, home, human...",[Lords wrong detainees - Straw Jack Straw atta...
4,3,18,3_aid_world_africa_brown,"[aid, world, africa, brown, g8, countries, sai...",['No UK apology ' colonial past days Britain a...
5,4,17,4_asylum_immigration_uk_howard,"[asylum, immigration, uk, howard, said, system...",[Clarke plans migrant point scheme Anyone plan...
6,5,15,5_hunting_ban_hunt_law,"[hunting, ban, hunt, law, hunts, said, dogs, p...",[Minister defends hunting ban law law banning ...
7,6,15,6_age_vote_election_electoral,"[age, vote, election, electoral, ballot, mock,...",[Parties warned 'grey vote' Political parties ...
8,7,15,7_eu_straw_constitution_china,"[eu, straw, constitution, china, referendum, t...",[Straw attacked China arms Moves lift European...
9,8,13,8_ukip_kilroysilk_party_veritas,"[ukip, kilroysilk, party, veritas, mr, robert,...",[Kilroy launches 'Veritas ' party Ex-BBC chat ...


In [37]:
# Get topic names
label_names = politics["model"].generate_topic_labels(nr_words=3, topic_prefix=True, word_length=7, separator='_')
# politics["model"].set_topic_labels(labels)
politics["model"].custom_labels_ = label_names
politics["model"].visualize_barchart(custom_labels=True)

In [40]:
politics["model"].visualize_barchart(topics=[0,1,2,4,5,6,7,12], custom_labels=True)

In [44]:
df = politics["model"].get_document_info(contents['politics'])
df.head()

,Document,Topic,Name,CustomName,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Election 'could terror target' Terrorists migh...,-1,-1_said_mr_would_people,-1_said_mr_would,"[said, mr, would, people, labour, government, ...",[Howard denies split ID cards Michael Howard d...,said - mr - would - people - labour - governme...,0.000000,False
1,'Debate needed ' donations cap cap donations p...,1,1_tax_labour_would_election,1_tax_labour_would,"[tax, labour, would, election, mr, brown, said...",[Howard dismisses Tory tax fears Michael Howar...,tax - labour - would - election - mr - brown -...,0.983589,False
2,Kennedy begins pre-election tour Liberal Democ...,-1,-1_said_mr_would_people,-1_said_mr_would,"[said, mr, would, people, labour, government, ...",[Howard denies split ID cards Michael Howard d...,said - mr - would - people - labour - governme...,0.000000,False
3,'Errors ' doomed first Dome sale initial attem...,-1,-1_said_mr_would_people,-1_said_mr_would,"[said, mr, would, people, labour, government, ...",[Howard denies split ID cards Michael Howard d...,said - mr - would - people - labour - governme...,0.000000,False
4,Correction agency plans dropped Plans create s...,2,2_said_police_law_would,2_said_police_law,"[said, police, law, would, rights, home, human...",[Lords wrong detainees - Straw Jack Straw atta...,said - police - law - would - rights - home - ...,0.666100,False


In [45]:
df.to_csv("politics_topics.csv", index=False)